## Python, SQL and defaultdict

In this notebook we're going to put a bunch of pieces together. 

* SQL: We'll build an "in memory" DB and write some complicated queries against it.
* `defaultdict`: building on the recorded lecture, we'll take the results of those complicated queries and store them in a sensible way in a `defaultdict`. We'll also practice writing these out to a file. 

This code contains a number of the techniques you need to do the _second_ part of the Wedge assignment. 

We start by reading in the file as before. Note the import of our `wedge_helper` library, which is stored in a file with that same name and the extension `py`. As before, open that file up (maybe in Spyder?) so that you can see what's inside.

In [1]:
import sqlite3
from pprint import pprint
from collections import defaultdict
from wedge_helper import *

input_file = "OwnerTransactions_30.txt"

# Let's just open the file and read the first few lines to the screen.
with open(input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split("\t"))
        if idx > 3 :
            break

['datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag', 'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id']
['2010-01-01 10:19:32', '6', '68', '2', '2.79DP9', 'GEN MERCH', 'D', ' ', ' ', '9', '2', '0', '0.0000', '2.7900', '5.5800', '2.7900', '0.0000', '1', '0', '0', '0', '0.0000', '0.0000', '3', '0', '0', '0.00000000', '2', '0', '0', '0.0000', '0', '0', '', '0', '0', '0', '0', '0', '0', 'NULL', '0', 'NULL', '', '0', '40825', '1', '0', '0', '1']
['2010-01-01 10:19:46', '6', '68', '2', '0000000

Let's do our standard open the DB in memory and create a cursor.

In [2]:
db = sqlite3.connect(':memory:') # SQLite makes it easy to build in-memory DBs
cur = db.cursor()

init_db(cur) # take a look at the .py file to see what happened here.

In [3]:
with open(input_file,'r',encoding="Latin-1") as ifile :
    populate_db(db,ifile,delimiter="\t",limit=None)

Because we're using functions from the helper file, what we just did looks deceptively simple. There's a full `transactions` table now stored in a DB in memory. And `cur` holds the connection to that DB.

In [7]:
cur.execute('SELECT * FROM transactions LIMIT 3')
cur.fetchall()

[('2010-01-01 10:19:32',
  6,
  68,
  2,
  '2.79DP9',
  'GEN MERCH',
  'D',
  ' ',
  ' ',
  9,
  2.0,
  0,
  0.0,
  2.79,
  5.58,
  2.79,
  0.0,
  1,
  0,
  0,
  0,
  0.0,
  0.0,
  3,
  0,
  0,
  0.0,
  2.0,
  0,
  0,
  0.0,
  0,
  0,
  '',
  0,
  0,
  0,
  0,
  '0',
  0,
  'NULL',
  0,
  'NULL',
  '',
  0,
  40825,
  1,
  0,
  0,
  1),
 ('2010-01-01 10:19:46',
  6,
  68,
  2,
  14240,
  'CEREAL Granola Vanilla Almond',
  'I',
  ' ',
  ' ',
  3,
  0.86,
  1,
  2.11,
  2.79,
  2.4,
  3.69,
  0.0,
  0,
  0,
  1,
  0,
  0.774,
  0.0,
  1,
  1,
  0,
  0.0,
  0.86,
  0,
  0,
  0.0,
  0,
  0,
  '',
  0,
  2,
  0,
  0,
  '0',
  0,
  'NULL',
  0,
  'NULL',
  '',
  0,
  40825,
  1,
  0,
  0,
  3),
 ('2010-01-01 10:19:50',
  6,
  68,
  2,
  4515701220,
  '2% Milk rBGH-free 64oz PoM',
  'I',
  ' ',
  ' ',
  4,
  1.0,
  0,
  1.95,
  2.39,
  2.39,
  2.39,
  0.0,
  0,
  0,
  1,
  0,
  0.0,
  0.0,
  1,
  0,
  0,
  0.0,
  1.0,
  0,
  0,
  0.0,
  0,
  0,
  '',
  0,
  1,
  0,
  0,
  '0',
  0,
  'NULL',


Notice how the query results come back. What sort of data type is it? What's the "inner" data type?

Again we'll run a "simple" query and print out the results in a semi-pretty fashion. Check out [this page](https://docs.python.org/3.6/library/string.html#format-specification-mini-language) to learn more about the formatting tricks.

In [8]:
result = cur.execute('''SELECT card_no,
                               date(datetime) as date,
                               sum(total) AS spend
                        FROM transactions
                        WHERE trans_type = "I"
                        GROUP BY card_no, date
                        ''')

for idx,row in enumerate(result) :
    print("On {1}, card_no = {0} spent {2:,.2f}.".format(row[0],row[1],row[2]))
    if idx > 20 :
        break

On 2010-01-03, card_no = 10179 spent 135.62.
On 2010-01-05, card_no = 10179 spent 25.82.
On 2010-01-06, card_no = 10179 spent 20.65.
On 2010-01-07, card_no = 10179 spent 4.79.
On 2010-01-09, card_no = 10179 spent 11.23.
On 2010-01-10, card_no = 10179 spent 155.29.
On 2010-01-13, card_no = 10179 spent 22.33.
On 2010-01-15, card_no = 10179 spent 40.66.
On 2010-01-17, card_no = 10179 spent 56.43.
On 2010-01-20, card_no = 10179 spent 40.68.
On 2010-01-22, card_no = 10179 spent 23.65.
On 2010-01-23, card_no = 10179 spent 5.29.
On 2010-01-24, card_no = 10179 spent 107.06.
On 2010-01-27, card_no = 10179 spent 13.76.
On 2010-02-03, card_no = 10179 spent 59.22.
On 2010-02-04, card_no = 10179 spent 71.62.
On 2010-02-09, card_no = 10179 spent 31.90.
On 2010-02-10, card_no = 10179 spent 38.94.
On 2010-02-14, card_no = 10179 spent 55.90.
On 2010-02-17, card_no = 10179 spent 32.17.
On 2010-02-18, card_no = 10179 spent 10.36.
On 2010-02-19, card_no = 10179 spent 8.03.


Obviously, in the real world you wouldn't want to just print this stuff to the screen, you'd store it. A dictionary, with keys of date and owner number, is a natural way to do that. As in the lecture, we'll do this first in a plain dictionary.

In [9]:
date_owner_spend = dict()

for idx,row in enumerate(result) :
    owner, date, spend = row
    spend = round(spend,2)
    
    if owner not in date_owner_spend :
        date_owner_spend[owner] = dict()
        date_owner_spend[owner][date] = spend
    else : 
        if date not in date_owner_spend[owner] :
            date_owner_spend[owner][date] = spend
        else :
            date_owner_spend[owner][date] += spend

In [10]:
# Let's look at one owner's spend
pprint(date_owner_spend[18736])

{'2011-02-15': 94.42,
 '2011-02-25': 151.03,
 '2011-03-01': 21.0,
 '2011-03-03': 105.71,
 '2011-03-04': 36.9,
 '2011-03-06': 31.81,
 '2011-03-09': 24.0,
 '2011-03-10': 25.0,
 '2011-03-13': 35.0,
 '2011-03-14': 15.0,
 '2011-03-20': 89.0,
 '2011-03-23': 45.31,
 '2011-03-27': 51.41,
 '2011-04-10': 54.45,
 '2011-04-12': 56.41,
 '2011-04-13': 8.0,
 '2011-04-14': 27.0,
 '2011-04-15': 32.53,
 '2011-04-17': 69.0,
 '2011-04-22': 124.01,
 '2011-04-23': 14.38,
 '2011-04-24': 37.71,
 '2011-04-26': 48.41,
 '2011-04-28': 5.59,
 '2011-04-30': 27.0,
 '2011-05-01': 102.66,
 '2011-05-03': 37.0,
 '2011-05-04': 11.72,
 '2011-05-07': 41.0,
 '2011-05-08': 71.12,
 '2011-05-12': 94.67,
 '2011-05-16': 23.0,
 '2011-05-18': 20.43,
 '2011-05-19': 14.96,
 '2011-05-23': 27.0,
 '2011-05-26': 70.82,
 '2011-05-27': 23.0,
 '2011-05-29': 39.41,
 '2011-05-30': 70.98,
 '2011-06-09': 27.73,
 '2011-06-13': 96.29,
 '2011-06-14': 28.0,
 '2011-06-17': 1.99,
 '2011-06-18': 75.55,
 '2011-06-26': 51.39,
 '2011-06-29': 36.8,
 '201

Let's do the same thing, but with `defaultdict`.

In [15]:
date_owner_spend_2 = defaultdict(lambda: defaultdict(float)) 

for idx,row in enumerate(result) :
    owner, date, spend = row
    spend = round(spend,2)
    
    date_owner_spend_2[owner][date] += spend
    

In [13]:
pprint(date_owner_spend[18736])

{'2011-02-15': 94.42,
 '2011-02-25': 151.03,
 '2011-03-01': 21.0,
 '2011-03-03': 105.71,
 '2011-03-04': 36.9,
 '2011-03-06': 31.81,
 '2011-03-09': 24.0,
 '2011-03-10': 25.0,
 '2011-03-13': 35.0,
 '2011-03-14': 15.0,
 '2011-03-20': 89.0,
 '2011-03-23': 45.31,
 '2011-03-27': 51.41,
 '2011-04-10': 54.45,
 '2011-04-12': 56.41,
 '2011-04-13': 8.0,
 '2011-04-14': 27.0,
 '2011-04-15': 32.53,
 '2011-04-17': 69.0,
 '2011-04-22': 124.01,
 '2011-04-23': 14.38,
 '2011-04-24': 37.71,
 '2011-04-26': 48.41,
 '2011-04-28': 5.59,
 '2011-04-30': 27.0,
 '2011-05-01': 102.66,
 '2011-05-03': 37.0,
 '2011-05-04': 11.72,
 '2011-05-07': 41.0,
 '2011-05-08': 71.12,
 '2011-05-12': 94.67,
 '2011-05-16': 23.0,
 '2011-05-18': 20.43,
 '2011-05-19': 14.96,
 '2011-05-23': 27.0,
 '2011-05-26': 70.82,
 '2011-05-27': 23.0,
 '2011-05-29': 39.41,
 '2011-05-30': 70.98,
 '2011-06-09': 27.73,
 '2011-06-13': 96.29,
 '2011-06-14': 28.0,
 '2011-06-17': 1.99,
 '2011-06-18': 75.55,
 '2011-06-26': 51.39,
 '2011-06-29': 36.8,
 '201

Now let's do a bigger query, like the one from last class. Let's get sales, transactions and items by department, year and month.

In [16]:
result = cur.execute('''SELECT department,
                               substr(date(datetime),1,4) AS year,
                               substr(date(datetime),6,2) AS month,
                               sum(total) AS spend,
                               count(distinct(date(datetime) || register_no ||
                                       emp_no || trans_no)) as Transactions,
                               sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                               FROM transactions
                               WHERE department != 0 and
                                    department != 15 and
                                    trans_status != 'M' and
                                    trans_status != 'C' and
                                    trans_status != 'J' and
                                   (trans_status = '' or 
                                    trans_status = ' ' or 
                                    trans_status = 'V' or 
                                    trans_status = 'R')
                          GROUP BY department, year, month
                          ORDER BY year, month''')

In [17]:
dept_ym = defaultdict(lambda: # dept_no
                      defaultdict(lambda: # year 
                                  defaultdict(lambda: # month
                                              defaultdict(float)))) # #field name and value
                      
for row in result :
    dept, year, month, spend, trans, items = row
    spend = round(spend,2)

    dept_ym[dept][year][month]['spend'] += spend
    dept_ym[dept][year][month]['trans'] += trans
    dept_ym[dept][year][month]['items'] += items

In [23]:
"-".join([str(item) for item in [dept,year,month]])

'18-2016-09'

Play around with this dictionary a bit to make sure you understand the structure.

Sometimes having the data in memory is all you need. Other times you're going to want to write it out to a file to work with further. Let's write out this dictionary.

In [ ]:
header = ["dept_no","year","month","spend","transactions","items"]

with open("dept_year_month.txt",'w') as ofile :
    ofile.write("\t".join(header) + "\n")
    
    for dept in dept_ym :
        for year in dept_ym[dept] :
            for month in dept_ym[dept][year] :
                row = [dept,year,month,
                      dept_ym[dept][year][month]['spend'],
                      dept_ym[dept][year][month]['trans'],
                      dept_ym[dept][year][month]['items']]
                
                ofile.write("\t".join([str(item) for item in row]) + "\n")
    

---

Now it's your turn. Write a query that builds the first task in Wedge Part 2: sales (and transactions and items) by date by hour. We haven't talked too much about how you get "hour" out of one of these transaction tables, so that's going to require some trial and error. One option for building queries: redo this notebook where the db is _not_ `:memory:` but where you actually give it a name. Then it will be on your computer, you can open your SQLite Browser and play around there. 

In [ ]:
result = cur.execute('''
                        
                        -- Your query here

                    ''')

Now store it in a `defaultdict` and write it out in the subsequent cell.

In [ ]:
# Store query results in a default dict

In [ ]:
# write out your default dict

Run the cell below as a best practice, but the DB will close if you quit the notebook. If it's an in-memory DB then it's also gone!

In [ ]:
db.close()